# 个贷违约预测

## 1. 数据预处理

### 1.1 导入数据

In [72]:
import numpy as np
import pandas as pd
#设置不隐藏列
pd.set_option('display.max_columns', None)
data = pd.read_csv('./data.csv')

### 1.2 查看数据

In [73]:
#查看前5行
data.head()

,loan_id,user_id,total_loan,year_of_loan,interest,monthly_payment,class,employer_type,industry,work_year,house_exist,issue_date,use,debt_loan_ratio,del_in_18month,known_outstanding_loan,known_dero,pub_dero_bankrup,early_return,early_return_amount,early_return_amount_3mon,isDefault
0,1040418,240418,31818.18182,3,11.466,1174.91,C,政府机构,金融业,3 years,0,2016/10/1,2,2.430000,0,3,0,0.0,3,9927,0.0,0
1,1025197,225197,28000.00000,5,16.841,670.69,C,政府机构,金融业,10+ years,0,2013/6/1,0,11.004545,1,3,0,0.0,0,0,0.0,0
2,1009360,209360,17272.72727,3,8.900,603.32,A,政府机构,公共服务、社会组织,10+ years,1,2014/1/1,4,6.409091,0,5,0,0.0,0,0,0.0,0
3,1039708,239708,20000.00000,3,4.788,602.30,A,世界五百强,文化和体育业,6 years,0,2015/7/1,0,9.204545,0,3,0,0.0,0,0,0.0,0
4,1027483,227483,15272.72727,3,12.790,470.31,C,政府机构,信息传输、软件和信息技术服务业,< 1 year,2,2016/7/1,0,15.578182,0,15,0,0.0,0,0,0.0,0


In [74]:
#数据维度
data.shape

(10000, 22)

In [75]:
#数据整体情况
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   loan_id                   10000 non-null  int64  
 1   user_id                   10000 non-null  int64  
 2   total_loan                10000 non-null  float64
 3   year_of_loan              10000 non-null  int64  
 4   interest                  10000 non-null  float64
 5   monthly_payment           10000 non-null  float64
 6   class                     10000 non-null  object 
 7   employer_type             10000 non-null  object 
 8   industry                  10000 non-null  object 
 9   work_year                 9378 non-null   object 
 10  house_exist               10000 non-null  int64  
 11  issue_date                10000 non-null  object 
 12  use                       10000 non-null  int64  
 13  debt_loan_ratio           10000 non-null  float64
 14  del_in_

In [76]:
#查看数据统计信息
data.describe()

,loan_id,user_id,total_loan,year_of_loan,interest,monthly_payment,house_exist,use,debt_loan_ratio,del_in_18month,known_outstanding_loan,known_dero,pub_dero_bankrup,early_return,early_return_amount,early_return_amount_3mon,isDefault
count,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,9993.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,1.025210e+06,225209.587700,14402.126591,3.479600,13.222782,436.960427,0.612200,1.762600,17.532170,0.311600,11.645000,0.226400,0.138997,1.290600,2173.916100,335.232240,0.168300
std,1.438682e+04,14386.820956,8953.946807,0.853965,4.875755,261.754396,0.672203,2.392031,14.218509,0.872457,5.500745,0.607931,0.379089,1.449469,3027.539977,635.108825,0.374151
min,1.000008e+06,200008.000000,818.181818,3.000000,4.779000,30.440000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.012973e+06,212973.250000,7500.000000,3.000000,9.702000,248.820000,0.000000,0.000000,11.157955,0.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.025276e+06,225276.500000,12272.727270,3.000000,12.639000,371.525000,1.000000,0.000000,16.652273,0.000000,11.000000,0.000000,0.000000,1.000000,838.500000,0.000000,0.000000
75%,1.037694e+06,237694.500000,19636.363640,3.000000,15.985500,573.830000,1.000000,4.000000,22.781591,0.000000,14.000000,0.000000,0.000000,3.000000,3354.250000,413.675000,0.000000
max,1.049997e+06,249997.000000,47272.727270,5.000000,33.979000,1503.890000,4.000000,13.000000,999.000000,15.000000,59.000000,12.000000,5.000000,5.000000,18413.000000,5523.900000,1.000000


### 1.3 检查数据质量

In [77]:
#查看数据缺失情况
data.isnull().sum()

loan_id                       0
user_id                       0
total_loan                    0
year_of_loan                  0
interest                      0
monthly_payment               0
class                         0
employer_type                 0
industry                      0
work_year                   622
house_exist                   0
issue_date                    0
use                           0
debt_loan_ratio               0
del_in_18month                0
known_outstanding_loan        0
known_dero                    0
pub_dero_bankrup              7
early_return                  0
early_return_amount           0
early_return_amount_3mon      0
isDefault                     0
dtype: int64

work_year与pub_dero_bankrup数据存在缺失，尝试缺失值补全。

### 1.4 数据清洗

### 1.5 离散属性编码

#### 需要编码的离散属性为class,employer_type,industry,work_year,issue_date

1.对class进行编码

In [78]:
#作出class与isDefault的列联表
class_anl = pd.crosstab(data['class'], data['isDefault'])
class_anl['ratio'] = class_anl.apply(lambda row: row[1] / row[0], axis=1)
print(class_anl.columns)
print(class_anl)

Index([0, 1, 'ratio'], dtype='object', name='isDefault')
isDefault     0    1     ratio
class                         
A          1597  101  0.063244
B          2617  348  0.132977
C          2303  556  0.241424
D          1139  344  0.302019
E           468  225  0.480769
F           155   86  0.554839
G            38   23  0.605263


In [79]:
#伴随贷款等级下降，违约率提升，故将class转化为序数属性
class_map = {
    'A': 0,
    'B': 1,
    'C': 2,
    'D': 3,
    'E': 4,
    'F': 5,
    'G': 6
}
data['class'] = data['class'].map(class_map)
data['class'].head()

0    2
1    2
2    0
3    0
4    2
Name: class, dtype: int64

2.对employer_type进行编码

In [80]:
data = data.join(pd.get_dummies(data['employer_type'], prefix ='employer_tpye', drop_first = True))
data.head()

,loan_id,user_id,total_loan,year_of_loan,interest,monthly_payment,class,employer_type,industry,work_year,house_exist,issue_date,use,debt_loan_ratio,del_in_18month,known_outstanding_loan,known_dero,pub_dero_bankrup,early_return,early_return_amount,early_return_amount_3mon,isDefault,employer_tpye_世界五百强,employer_tpye_幼教与中小学校,employer_tpye_政府机构,employer_tpye_普通企业,employer_tpye_高等教育机构
0,1040418,240418,31818.18182,3,11.466,1174.91,2,政府机构,金融业,3 years,0,2016/10/1,2,2.430000,0,3,0,0.0,3,9927,0.0,0,False,False,True,False,False
1,1025197,225197,28000.00000,5,16.841,670.69,2,政府机构,金融业,10+ years,0,2013/6/1,0,11.004545,1,3,0,0.0,0,0,0.0,0,False,False,True,False,False
2,1009360,209360,17272.72727,3,8.900,603.32,0,政府机构,公共服务、社会组织,10+ years,1,2014/1/1,4,6.409091,0,5,0,0.0,0,0,0.0,0,False,False,True,False,False
3,1039708,239708,20000.00000,3,4.788,602.30,0,世界五百强,文化和体育业,6 years,0,2015/7/1,0,9.204545,0,3,0,0.0,0,0,0.0,0,True,False,False,False,False
4,1027483,227483,15272.72727,3,12.790,470.31,2,政府机构,信息传输、软件和信息技术服务业,< 1 year,2,2016/7/1,0,15.578182,0,15,0,0.0,0,0,0.0,0,False,False,True,False,False


3.对industry进行编码

In [81]:
data = data.join(pd.get_dummies(data['industry'], prefix ='industry', drop_first = True))
data.head()

,loan_id,user_id,total_loan,year_of_loan,interest,monthly_payment,class,employer_type,industry,work_year,house_exist,issue_date,use,debt_loan_ratio,del_in_18month,known_outstanding_loan,known_dero,pub_dero_bankrup,early_return,early_return_amount,early_return_amount_3mon,isDefault,employer_tpye_世界五百强,employer_tpye_幼教与中小学校,employer_tpye_政府机构,employer_tpye_普通企业,employer_tpye_高等教育机构,industry_住宿和餐饮业,industry_信息传输、软件和信息技术服务业,industry_公共服务、社会组织,industry_农、林、牧、渔业,industry_制造业,industry_国际组织,industry_建筑业,industry_房地产业,industry_批发和零售业,industry_文化和体育业,industry_电力、热力生产供应业,industry_采矿业,industry_金融业
0,1040418,240418,31818.18182,3,11.466,1174.91,2,政府机构,金融业,3 years,0,2016/10/1,2,2.430000,0,3,0,0.0,3,9927,0.0,0,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
1,1025197,225197,28000.00000,5,16.841,670.69,2,政府机构,金融业,10+ years,0,2013/6/1,0,11.004545,1,3,0,0.0,0,0,0.0,0,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
2,1009360,209360,17272.72727,3,8.900,603.32,0,政府机构,公共服务、社会组织,10+ years,1,2014/1/1,4,6.409091,0,5,0,0.0,0,0,0.0,0,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
3,1039708,239708,20000.00000,3,4.788,602.30,0,世界五百强,文化和体育业,6 years,0,2015/7/1,0,9.204545,0,3,0,0.0,0,0,0.0,0,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
4,1027483,227483,15272.72727,3,12.790,470.31,2,政府机构,信息传输、软件和信息技术服务业,< 1 year,2,2016/7/1,0,15.578182,0,15,0,0.0,0,0,0.0,0,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False


4.对work_year进行编码

In [82]:
def convert_work(work_year):
    work_year=str(work_year).strip()
    # < 1 year 转为 0
    if work_year == '< 1 year':
        return 0
    # 10+ years 转为 11
    elif work_year == '10+ years':
        return 11
    # 其余年份提取年份数值
    else:
        try:
            return int(work_year.split()[0])
        except (ValueError):
            return np.nan
data['work_year'] = data['work_year'].apply(convert_work)
data.head()

,loan_id,user_id,total_loan,year_of_loan,interest,monthly_payment,class,employer_type,industry,work_year,house_exist,issue_date,use,debt_loan_ratio,del_in_18month,known_outstanding_loan,known_dero,pub_dero_bankrup,early_return,early_return_amount,early_return_amount_3mon,isDefault,employer_tpye_世界五百强,employer_tpye_幼教与中小学校,employer_tpye_政府机构,employer_tpye_普通企业,employer_tpye_高等教育机构,industry_住宿和餐饮业,industry_信息传输、软件和信息技术服务业,industry_公共服务、社会组织,industry_农、林、牧、渔业,industry_制造业,industry_国际组织,industry_建筑业,industry_房地产业,industry_批发和零售业,industry_文化和体育业,industry_电力、热力生产供应业,industry_采矿业,industry_金融业
0,1040418,240418,31818.18182,3,11.466,1174.91,2,政府机构,金融业,3.0,0,2016/10/1,2,2.430000,0,3,0,0.0,3,9927,0.0,0,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
1,1025197,225197,28000.00000,5,16.841,670.69,2,政府机构,金融业,11.0,0,2013/6/1,0,11.004545,1,3,0,0.0,0,0,0.0,0,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
2,1009360,209360,17272.72727,3,8.900,603.32,0,政府机构,公共服务、社会组织,11.0,1,2014/1/1,4,6.409091,0,5,0,0.0,0,0,0.0,0,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
3,1039708,239708,20000.00000,3,4.788,602.30,0,世界五百强,文化和体育业,6.0,0,2015/7/1,0,9.204545,0,3,0,0.0,0,0,0.0,0,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
4,1027483,227483,15272.72727,3,12.790,470.31,2,政府机构,信息传输、软件和信息技术服务业,0.0,2,2016/7/1,0,15.578182,0,15,0,0.0,0,0,0.0,0,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False


5.对issue_date进行编码

In [86]:
data['issue_date'] = pd.to_datetime(data['issue_date'])
data['issue_year'] = data['issue_date'].dt.year
data['issue_month'] = data['issue_date'].dt.month
data.head()

,loan_id,user_id,total_loan,year_of_loan,interest,monthly_payment,class,employer_type,industry,work_year,house_exist,issue_date,use,debt_loan_ratio,del_in_18month,known_outstanding_loan,known_dero,pub_dero_bankrup,early_return,early_return_amount,early_return_amount_3mon,isDefault,employer_tpye_世界五百强,employer_tpye_幼教与中小学校,employer_tpye_政府机构,employer_tpye_普通企业,employer_tpye_高等教育机构,industry_住宿和餐饮业,industry_信息传输、软件和信息技术服务业,industry_公共服务、社会组织,industry_农、林、牧、渔业,industry_制造业,industry_国际组织,industry_建筑业,industry_房地产业,industry_批发和零售业,industry_文化和体育业,industry_电力、热力生产供应业,industry_采矿业,industry_金融业,issue_year,issue_month
0,1040418,240418,31818.18182,3,11.466,1174.91,2,政府机构,金融业,3.0,0,2016-10-01,2,2.430000,0,3,0,0.0,3,9927,0.0,0,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,2016,10
1,1025197,225197,28000.00000,5,16.841,670.69,2,政府机构,金融业,11.0,0,2013-06-01,0,11.004545,1,3,0,0.0,0,0,0.0,0,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,2013,6
2,1009360,209360,17272.72727,3,8.900,603.32,0,政府机构,公共服务、社会组织,11.0,1,2014-01-01,4,6.409091,0,5,0,0.0,0,0,0.0,0,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,2014,1
3,1039708,239708,20000.00000,3,4.788,602.30,0,世界五百强,文化和体育业,6.0,0,2015-07-01,0,9.204545,0,3,0,0.0,0,0,0.0,0,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,2015,7
4,1027483,227483,15272.72727,3,12.790,470.31,2,政府机构,信息传输、软件和信息技术服务业,0.0,2,2016-07-01,0,15.578182,0,15,0,0.0,0,0,0.0,0,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,2016,7
